**Get the intersection of the outliers detected by the number of models**

In [1]:
import os
import pandas as pd
import pyodbc

# Directory structure where the outlier results are stored
base_dir = "Outlier_Detection_Test_8/Transaction_Level/Union_of_Outliers"
summary_output_dir = "Outlier_Detection_Test_8"  # Directory for saving the summary table

# Create directories if they don't exist
os.makedirs(summary_output_dir, exist_ok=True)

# Initialize a list to store summary information
summary = []

# Step 1: Get the list of combined outlier files from the Union_of_Outliers directory
combined_files = [f for f in os.listdir(base_dir) if f.endswith('_Combined_Outliers.csv')]

# Step 2: Process each combined file and calculate the summary statistics
for file in combined_files:
    # Extract fact table name from the filename
    table_name = file.replace('_Combined_Outliers.csv', '')
    
    # Load the combined outlier data
    file_path = os.path.join(base_dir, file)
    combined_data = pd.read_csv(file_path)
    
    # Count how many models flagged each row as an outlier (-1 for LOF, IF, OCSVM and Autoencoder)
    # Create a new column 'Outlier_Count' that counts how many models detected an outlier
    combined_data['Outlier_Count'] = (
        (combined_data['LOF_Outlier'] == -1).astype(int) +
        (combined_data['IForest_Outlier'] == -1).astype(int) +
        (combined_data['OCSVM_Outlier'] == -1).astype(int) +
        (combined_data['Autoencoder_Outlier'] == -1).astype(int)
    )
    
    # Count the number of rows where:
    # - 4 models detected an outlier
    count_4_models = (combined_data['Outlier_Count'] == 4).sum()
    
    # - At least 3 models detected an outlier
    count_3_models = (combined_data['Outlier_Count'] >= 3).sum()
    
    # - At least 2 models detected an outlier
    count_2_models = (combined_data['Outlier_Count'] >= 2).sum()
    
    # - At least 1 model detected an outlier
    count_1_model = (combined_data['Outlier_Count'] >= 1).sum()
    
    # Append the results for this fact table to the summary list
    summary.append({
        'Fact Table': table_name,
        'Detected by 4 Models': count_4_models,
        'Detected by at least 3 Models': count_3_models,
        'Detected by at least 2 Models': count_2_models,
        'Detected by at least 1 Model': count_1_model
    })

# Step 3: Convert the summary list to a DataFrame
summary_df = pd.DataFrame(summary)

# Step 4: Save the summary DataFrame as a CSV file
summary_output_path = os.path.join(summary_output_dir, 'Outlier_Summary.csv')
summary_df.to_csv(summary_output_path, index=False)

print(f"Summary table saved to {summary_output_path}")

# Display the summary table
print(summary_df)


Summary table saved to Outlier_Detection_Test_8\Outlier_Summary.csv
             Fact Table  Detected by 4 Models  Detected by at least 3 Models  \
0        FactCallCenter                     0                              2   
1      FactCurrencyRate                     0                             40   
2           FactFinance                     0                            206   
3     FactInternetSales                     0                             21   
4  FactProductInventory                     0                           5229   
5     FactResellerSales                    35                            276   
6        FactSalesQuota                     0                              0   

   Detected by at least 2 Models  Detected by at least 1 Model  
0                              4                            17  
1                            194                           955  
2                            786                          2360  
3                           201

In [2]:
summary_df

,Fact Table,Detected by 4 Models,Detected by at least 3 Models,Detected by at least 2 Models,Detected by at least 1 Model
0,FactCallCenter,0,2,4,17
1,FactCurrencyRate,0,40,194,955
2,FactFinance,0,206,786,2360
3,FactInternetSales,0,21,2016,4754
4,FactProductInventory,0,5229,18828,68672
5,FactResellerSales,35,276,879,3634
6,FactSalesQuota,0,0,3,23


**Using a threshold value, select the final outliers**

In [7]:
import os
import pandas as pd

# Directory structure where the outlier results are stored
base_dir = "Outlier_Detection_Test_8/Transaction_Level/Union_of_Outliers"
final_outlier_dir = "Outlier_Detection_Test_9/Final_Outliers"  # Directory for saving the final outliers

# Create directories if they don't exist
os.makedirs(final_outlier_dir, exist_ok=True)

# Load the summary table
summary_output_path = "Outlier_Detection_Test_8/Outlier_Summary.csv"
summary_df = pd.read_csv(summary_output_path)

# Iterate through each fact table, load its combined outlier dataset, and select final outliers
for index, row in summary_df.iterrows():
    fact_table = row['Fact Table']
    
    # Load the combined outlier dataset for the current fact table
    combined_outlier_path = os.path.join(base_dir, f"{fact_table}_Combined_Outliers.csv")
    combined_df = pd.read_csv(combined_outlier_path)
    
    # Count how many models detected an outlier for each transaction
    combined_df['Outlier_Count'] = (
        (combined_df['LOF_Outlier'] == -1).astype(int) +
        (combined_df['IForest_Outlier'] == -1).astype(int) +
        (combined_df['OCSVM_Outlier'] == -1).astype(int) +
        (combined_df['Autoencoder_Outlier'] == -1).astype(int)
    )
    
    # Determine the threshold based on the presence of outliers detected by different numbers of models
    if (combined_df['Outlier_Count'] == 4).any():       # Check if there are any outliers detected by all 4 models
        threshold = 4
    elif (combined_df['Outlier_Count'] >= 3).any():     # Check if there are any outliers detected by at least 3 models
        threshold = 3
    elif (combined_df['Outlier_Count'] >= 2).any():     # Check if there are any outliers detected by at least 2 models
        threshold = 2
    else:
        threshold = 1
    
    # Mark the final outliers based on the threshold
    combined_df['Final_Outlier'] = combined_df['Outlier_Count'].apply(lambda x: 1 if x >= threshold else 0)
    
    # Add the threshold value as a column in the dataset
    combined_df['Threshold'] = threshold
    
    # Save the final outliers to a new CSV file
    final_outlier_output_path = os.path.join(final_outlier_dir, f"{fact_table}_Final_Outliers.csv")
    combined_df.to_csv(final_outlier_output_path, index=False)
    
    print(f"Final outliers for {fact_table} saved to {final_outlier_output_path} with threshold {threshold}")


Final outliers for FactCallCenter saved to Outlier_Detection_Test_9/Final_Outliers\FactCallCenter_Final_Outliers.csv with threshold 3
Final outliers for FactCurrencyRate saved to Outlier_Detection_Test_9/Final_Outliers\FactCurrencyRate_Final_Outliers.csv with threshold 3
Final outliers for FactFinance saved to Outlier_Detection_Test_9/Final_Outliers\FactFinance_Final_Outliers.csv with threshold 3
Final outliers for FactInternetSales saved to Outlier_Detection_Test_9/Final_Outliers\FactInternetSales_Final_Outliers.csv with threshold 3
Final outliers for FactProductInventory saved to Outlier_Detection_Test_9/Final_Outliers\FactProductInventory_Final_Outliers.csv with threshold 3
Final outliers for FactResellerSales saved to Outlier_Detection_Test_9/Final_Outliers\FactResellerSales_Final_Outliers.csv with threshold 4
Final outliers for FactSalesQuota saved to Outlier_Detection_Test_9/Final_Outliers\FactSalesQuota_Final_Outliers.csv with threshold 2


In [10]:
import os
import pandas as pd

# Directory structure where the outlier results are stored
base_dir = "Outlier_Detection_Test_8/Transaction_Level/Union_of_Outliers"
final_outlier_dir = "Outlier_Detection_Test_9/Final_Outliers"  # Directory for saving the final outliers
only_final_outliers_dir = "Outlier_Detection_Test_9/Only_Final_Outliers"  # Directory to store only final outliers for analysis

# Create directories if they don't exist
os.makedirs(final_outlier_dir, exist_ok=True)
os.makedirs(only_final_outliers_dir, exist_ok=True)

# Load the summary table
summary_output_path = "Outlier_Detection_Test_8/Outlier_Summary.csv"
summary_df = pd.read_csv(summary_output_path)

# Iterate through each fact table, load its combined outlier dataset, and select final outliers
for index, row in summary_df.iterrows():
    fact_table = row['Fact Table']
    
    # Load the combined outlier dataset for the current fact table
    combined_outlier_path = os.path.join(base_dir, f"{fact_table}_Combined_Outliers.csv")
    combined_df = pd.read_csv(combined_outlier_path)
    
    # Count how many models detected an outlier for each transaction
    combined_df['Outlier_Count'] = (
        (combined_df['LOF_Outlier'] == -1).astype(int) +
        (combined_df['IForest_Outlier'] == -1).astype(int) +
        (combined_df['OCSVM_Outlier'] == -1).astype(int) +
        (combined_df['Autoencoder_Outlier'] == -1).astype(int)
    )
    
    # Determine the threshold based on the presence of outliers detected by different numbers of models
    if (combined_df['Outlier_Count'] == 4).any():       # Check if there are any outliers detected by all 4 models
        threshold = 4
    elif (combined_df['Outlier_Count'] >= 3).any():     # Check if there are any outliers detected by at least 3 models
        threshold = 3
    elif (combined_df['Outlier_Count'] >= 2).any():     # Check if there are any outliers detected by at least 2 models
        threshold = 2
    else:
        threshold = 1
    
    # Mark the final outliers based on the threshold
    combined_df['Final_Outlier'] = combined_df['Outlier_Count'].apply(lambda x: 1 if x >= threshold else 0)
    
    # Add the threshold value as a column in the dataset
    combined_df['Threshold'] = threshold
    
    # Save the final outliers to a new CSV file
    final_outlier_output_path = os.path.join(final_outlier_dir, f"{fact_table}_Final_Outliers.csv")
    combined_df.to_csv(final_outlier_output_path, index=False)
    
    # Extract only the rows where 'Final_Outlier' is 1 (i.e., only final outliers)
    only_final_outliers_df = combined_df[combined_df['Final_Outlier'] == 1]
    
    # Save only the final outliers to a new CSV file for analysis
    only_final_outliers_output_path = os.path.join(only_final_outliers_dir, f"{fact_table}_Only_Final_Outliers.csv")
    only_final_outliers_df.to_csv(only_final_outliers_output_path, index=False)
    
    print(f"Final outliers for {fact_table} saved to {final_outlier_output_path} with threshold {threshold}")
    print(f"Only final outliers for {fact_table} saved to {only_final_outliers_output_path}")


Final outliers for FactCallCenter saved to Outlier_Detection_Test_9/Final_Outliers\FactCallCenter_Final_Outliers.csv with threshold 3
Only final outliers for FactCallCenter saved to Outlier_Detection_Test_9/Only_Final_Outliers\FactCallCenter_Only_Final_Outliers.csv
Final outliers for FactCurrencyRate saved to Outlier_Detection_Test_9/Final_Outliers\FactCurrencyRate_Final_Outliers.csv with threshold 3
Only final outliers for FactCurrencyRate saved to Outlier_Detection_Test_9/Only_Final_Outliers\FactCurrencyRate_Only_Final_Outliers.csv
Final outliers for FactFinance saved to Outlier_Detection_Test_9/Final_Outliers\FactFinance_Final_Outliers.csv with threshold 3
Only final outliers for FactFinance saved to Outlier_Detection_Test_9/Only_Final_Outliers\FactFinance_Only_Final_Outliers.csv
Final outliers for FactInternetSales saved to Outlier_Detection_Test_9/Final_Outliers\FactInternetSales_Final_Outliers.csv with threshold 3
Only final outliers for FactInternetSales saved to Outlier_Detecti

**Final Outliers**

In [11]:
import pandas as pd
import os

# Specify the folder path containing the CSV files
folder_path = 'Outlier_Detection_Test_9/Only_Final_Outliers'  # Replace with the path to your folder

# List to store file names and row counts
file_info = []

# Loop through each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        # Load the CSV file
        file_path = os.path.join(folder_path, file_name)
        data = pd.read_csv(file_path)
        
        # Get the row count and store the information
        row_count = len(data)
        file_info.append({'File Name': file_name, 'Row Count': row_count})

# Convert the list to a DataFrame and print it
file_info_df = pd.DataFrame(file_info)
print(file_info_df)


                                      File Name  Row Count
0        FactCallCenter_Only_Final_Outliers.csv          2
1      FactCurrencyRate_Only_Final_Outliers.csv         40
2           FactFinance_Only_Final_Outliers.csv        206
3     FactInternetSales_Only_Final_Outliers.csv         21
4  FactProductInventory_Only_Final_Outliers.csv       5229
5     FactResellerSales_Only_Final_Outliers.csv         35
6        FactSalesQuota_Only_Final_Outliers.csv          3


**Aggregation level one**

In [1]:
import pyodbc
import pandas as pd
import os
import re

# Step 1: Connect to SQL Server
try:
    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=DESKTOP-J7VSVLC\\SQLEXPRESS;'
                          'Database=AdventureWorksDW2019(1);'
                          'Trusted_Connection=yes;')
    print("Connected Successfully")
except Exception as e:
    print(f"Connection failed: {e}")
    conn = None

# Proceed only if connection is successful
if conn:
    # Base directory for final outliers
    all_final_outliers_dir = "Outlier_Detection_Test_9/Only_Final_Outliers"
    
    # Directory for aggregated results
    aggregated_dir = "Outlier_Detection_Test_9/Aggregated_By_One_Category"
    os.makedirs(aggregated_dir, exist_ok=True)

    # Step 2: Identify dimension tables and their categorical columns
    dim_tables_query = """
    SELECT TABLE_NAME, COLUMN_NAME
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME IN (
        SELECT TABLE_NAME 
        FROM INFORMATION_SCHEMA.TABLES 
        WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_NAME LIKE 'Dim%'
    )
    AND DATA_TYPE IN ('varchar', 'nvarchar', 'char', 'text')
    """
    
    dim_columns_df = pd.read_sql(dim_tables_query, conn)
    
    # Step 3: Filter out non-English columns and columns containing specified substrings
    non_english_pattern = re.compile(r'[^\x00-\x7F]+')
    
    # Function to check if a column name contains non-English characters or specified substrings
    def is_valid_column(column_name):
        substrings_to_remove = ['Description', 'FirstName', 'MiddleName', 'PhoneNumber', 'Phone', 'LastName', 'Address', 'ID', 'Key']
        if non_english_pattern.search(column_name):
            return False
        if any(substring in column_name for substring in substrings_to_remove):
            return False
        return True

    # Filter valid columns
    dim_columns_df = dim_columns_df[dim_columns_df['COLUMN_NAME'].apply(is_valid_column)]

    # Step 4: Process each fact table and dynamically join with dimension tables
    for file in os.listdir(all_final_outliers_dir):
        if file.endswith("_Only_Final_Outliers.csv"):
            # Extract the fact table name from the file
            table = file.replace("_Only_Final_Outliers.csv", "")
            agreed_outlier_file_path = os.path.join(all_final_outliers_dir, file)
            
            if os.path.exists(agreed_outlier_file_path):
                # Load the final outliers CSV file
                outlier_data = pd.read_csv(agreed_outlier_file_path)

                # Dynamically join outlier data with each dimension table based on foreign key
                for dim_table, dim_column in dim_columns_df.groupby('TABLE_NAME'):
                    categorical_columns = dim_column['COLUMN_NAME'].tolist()
                    
                    # Generate the query to get the dimension data
                    dim_data_query = f"SELECT * FROM {dim_table}"
                    dim_data = pd.read_sql(dim_data_query, conn)

                    # Determine the common foreign key column
                    foreign_keys = set(outlier_data.columns).intersection(set(dim_data.columns))
                    if foreign_keys:
                        foreign_key = foreign_keys.pop()
                        
                        # Merge outlier data with dimension table data on the foreign key
                        merged_data = pd.merge(outlier_data, dim_data, left_on=foreign_key, right_on=foreign_key, how='inner')

                        # Group by the categorical columns
                        for cat_column in categorical_columns:
                            aggregated_data = merged_data.groupby(cat_column).size().reset_index(name='OutlierCount')

                            # Save aggregated results to CSV
                            aggregated_output_path = os.path.join(aggregated_dir, f"{table}_Aggregated_By_{dim_table}_{cat_column}.csv")
                            aggregated_data.to_csv(aggregated_output_path, index=False)
                            print(f"Stored aggregated results for {table} by {dim_table}.{cat_column} in {aggregated_output_path}.")
                    else:
                        print(f"No common foreign key found for table {table} and dimension table {dim_table}. Skipping...")
            else:
                print(f"No agreed outliers file found for {table}. Skipping...")

    # Close the database connection
    conn.close()
else:
    print("Database connection failed.")


Connected Successfully


C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1494919461.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_columns_df = pd.read_sql(dim_tables_query, conn)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1494919461.py:72: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data = pd.read_sql(dim_data_query, conn)


No common foreign key found for table FactCallCenter and dimension table DimAccount. Skipping...
No common foreign key found for table FactCallCenter and dimension table DimCurrency. Skipping...
No common foreign key found for table FactCallCenter and dimension table DimCustomer. Skipping...
Stored aggregated results for FactCallCenter by DimDate.EnglishDayNameOfWeek in Outlier_Detection_Test_9/Aggregated_By_One_Category\FactCallCenter_Aggregated_By_DimDate_EnglishDayNameOfWeek.csv.
Stored aggregated results for FactCallCenter by DimDate.SpanishDayNameOfWeek in Outlier_Detection_Test_9/Aggregated_By_One_Category\FactCallCenter_Aggregated_By_DimDate_SpanishDayNameOfWeek.csv.
Stored aggregated results for FactCallCenter by DimDate.FrenchDayNameOfWeek in Outlier_Detection_Test_9/Aggregated_By_One_Category\FactCallCenter_Aggregated_By_DimDate_FrenchDayNameOfWeek.csv.
Stored aggregated results for FactCallCenter by DimDate.EnglishMonthName in Outlier_Detection_Test_9/Aggregated_By_One_Categ

C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1494919461.py:72: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data = pd.read_sql(dim_data_query, conn)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1494919461.py:72: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data = pd.read_sql(dim_data_query, conn)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1494919461.py:72: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data = pd.read_sql(dim_data_query, conn)


No common foreign key found for table FactCallCenter and dimension table DimReseller. Skipping...
No common foreign key found for table FactCallCenter and dimension table DimSalesReason. Skipping...
No common foreign key found for table FactCallCenter and dimension table DimSalesTerritory. Skipping...
No common foreign key found for table FactCallCenter and dimension table DimScenario. Skipping...
No common foreign key found for table FactCurrencyRate and dimension table DimAccount. Skipping...
Stored aggregated results for FactCurrencyRate by DimCurrency.CurrencyName in Outlier_Detection_Test_9/Aggregated_By_One_Category\FactCurrencyRate_Aggregated_By_DimCurrency_CurrencyName.csv.
No common foreign key found for table FactCurrencyRate and dimension table DimCustomer. Skipping...
Stored aggregated results for FactCurrencyRate by DimDate.EnglishDayNameOfWeek in Outlier_Detection_Test_9/Aggregated_By_One_Category\FactCurrencyRate_Aggregated_By_DimDate_EnglishDayNameOfWeek.csv.
Stored agg

...

C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1494919461.py:72: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data = pd.read_sql(dim_data_query, conn)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1494919461.py:72: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data = pd.read_sql(dim_data_query, conn)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1494919461.py:72: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data = pd.read_sql(dim_data_query, conn)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\149491

No common foreign key found for table FactResellerSales and dimension table DimCustomer. Skipping...
No common foreign key found for table FactResellerSales and dimension table DimDate. Skipping...
No common foreign key found for table FactResellerSales and dimension table DimDepartmentGroup. Skipping...
Stored aggregated results for FactResellerSales by DimEmployee.Title in Outlier_Detection_Test_9/Aggregated_By_One_Category\FactResellerSales_Aggregated_By_DimEmployee_Title.csv.
Stored aggregated results for FactResellerSales by DimEmployee.EmergencyContactName in Outlier_Detection_Test_9/Aggregated_By_One_Category\FactResellerSales_Aggregated_By_DimEmployee_EmergencyContactName.csv.
Stored aggregated results for FactResellerSales by DimEmployee.DepartmentName in Outlier_Detection_Test_9/Aggregated_By_One_Category\FactResellerSales_Aggregated_By_DimEmployee_DepartmentName.csv.
Stored aggregated results for FactResellerSales by DimEmployee.Status in Outlier_Detection_Test_9/Aggregated_

C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1494919461.py:72: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data = pd.read_sql(dim_data_query, conn)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1494919461.py:72: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data = pd.read_sql(dim_data_query, conn)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1494919461.py:72: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data = pd.read_sql(dim_data_query, conn)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\149491

No common foreign key found for table FactSalesQuota and dimension table DimCustomer. Skipping...
Stored aggregated results for FactSalesQuota by DimDate.EnglishDayNameOfWeek in Outlier_Detection_Test_9/Aggregated_By_One_Category\FactSalesQuota_Aggregated_By_DimDate_EnglishDayNameOfWeek.csv.
Stored aggregated results for FactSalesQuota by DimDate.SpanishDayNameOfWeek in Outlier_Detection_Test_9/Aggregated_By_One_Category\FactSalesQuota_Aggregated_By_DimDate_SpanishDayNameOfWeek.csv.
Stored aggregated results for FactSalesQuota by DimDate.FrenchDayNameOfWeek in Outlier_Detection_Test_9/Aggregated_By_One_Category\FactSalesQuota_Aggregated_By_DimDate_FrenchDayNameOfWeek.csv.
Stored aggregated results for FactSalesQuota by DimDate.EnglishMonthName in Outlier_Detection_Test_9/Aggregated_By_One_Category\FactSalesQuota_Aggregated_By_DimDate_EnglishMonthName.csv.
Stored aggregated results for FactSalesQuota by DimDate.SpanishMonthName in Outlier_Detection_Test_9/Aggregated_By_One_Category\Fact

C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1494919461.py:72: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data = pd.read_sql(dim_data_query, conn)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1494919461.py:72: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data = pd.read_sql(dim_data_query, conn)


**Aggregation level two**

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Directory where aggregated results are stored
aggregated_dir = "Outlier_Detection_Test_9/Aggregated_By_One_Category"

# Directory for visualization results
visualization_dir = "Outlier_Detection_Test_9/Visualizations"
os.makedirs(visualization_dir, exist_ok=True)

# Loop through the aggregated CSV files in the directory
for file_name in os.listdir(aggregated_dir):
    if file_name.endswith(".csv"):
        # Read the aggregated data
        file_path = os.path.join(aggregated_dir, file_name)
        aggregated_data = pd.read_csv(file_path)

        # Ensure DataFrame is not empty
        if aggregated_data.empty:
            print(f"DataFrame is empty for file: {file_name}. Skipping visualization.")
            continue
        
        # Extract table and dimension names for labeling
        parts = file_name.replace('.csv', '').split('_')
        table_name = parts[0]
        dim_table_name = parts[2]

        # Identify the categorical column (not 'OutlierCount')
        cat_columns = aggregated_data.columns.difference(['OutlierCount'])
        
        # Ensure there is a categorical column to plot
        if len(cat_columns) == 0:
            print(f"No categorical columns found in DataFrame for file: {file_name}. Skipping visualization.")
            continue
        
        cat_column_name = cat_columns[0]

        # Ensure the columns have valid data for plotting
        if aggregated_data[cat_column_name].isnull().all() or aggregated_data['OutlierCount'].isnull().all():
            print(f"Data for plotting is invalid or missing in file: {file_name}. Skipping visualization.")
            continue

        # Set figure size larger for better readability
        plt.figure(figsize=(20, 10))  # Increased figure size for clarity

        # Visualization: Bar plot of Outlier Count by Category
        sns.barplot(x=cat_column_name, y='OutlierCount', data=aggregated_data)
        plt.title(f'Outlier Count by {cat_column_name} for {table_name}')
        
        # Adjust x-axis for better readability
        plt.xticks(rotation=45, ha='right')  # Rotate x labels for better spacing
        plt.tight_layout()

        # Optionally limit the number of ticks on the x-axis if too crowded
        if len(aggregated_data[cat_column_name].unique()) > 20:  # Adjust this threshold as needed
            plt.xticks(ticks=range(0, len(aggregated_data[cat_column_name]), len(aggregated_data[cat_column_name]) // 20))

        # Save the plot
        plot_output_path = os.path.join(visualization_dir, f"{table_name}_Outliers_By_{dim_table_name}_{cat_column_name}.png")
        plt.savefig(plot_output_path)
        plt.close()
        print(f"Saved visualization for {table_name} by {dim_table_name}.{cat_column_name} in {plot_output_path}.")

print("Visualizations have been generated and saved successfully.")


Saved visualization for FactCallCenter by By.EnglishDayNameOfWeek in Outlier_Detection_Test_9/Visualizations\FactCallCenter_Outliers_By_By_EnglishDayNameOfWeek.png.
Saved visualization for FactCallCenter by By.EnglishMonthName in Outlier_Detection_Test_9/Visualizations\FactCallCenter_Outliers_By_By_EnglishMonthName.png.
Saved visualization for FactCallCenter by By.FrenchDayNameOfWeek in Outlier_Detection_Test_9/Visualizations\FactCallCenter_Outliers_By_By_FrenchDayNameOfWeek.png.
Saved visualization for FactCallCenter by By.FrenchMonthName in Outlier_Detection_Test_9/Visualizations\FactCallCenter_Outliers_By_By_FrenchMonthName.png.
Saved visualization for FactCallCenter by By.SpanishDayNameOfWeek in Outlier_Detection_Test_9/Visualizations\FactCallCenter_Outliers_By_By_SpanishDayNameOfWeek.png.
Saved visualization for FactCallCenter by By.SpanishMonthName in Outlier_Detection_Test_9/Visualizations\FactCallCenter_Outliers_By_By_SpanishMonthName.png.
Saved visualization for FactCurrencyRa

**Aggregation Level Three**

In [3]:
import pyodbc
import pandas as pd
import os
import re
from itertools import combinations

# Step 1: Connect to SQL Server
try:
    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=DESKTOP-J7VSVLC\\SQLEXPRESS;'
                          'Database=AdventureWorksDW2019(1);'
                          'Trusted_Connection=yes;')
    print("Connected Successfully")
except Exception as e:
    print(f"Connection failed: {e}")
    conn = None

# Proceed only if connection is successful
if conn:
    # Base directory for final outliers
    all_final_outliers_dir = "Outlier_Detection_Test_9/Only_Final_Outliers"
    
    # Directory for aggregated results
    aggregated_dir = "Outlier_Detection_Test_9/Aggregated_By_Two_Category"
    os.makedirs(aggregated_dir, exist_ok=True)

    # Step 2: Identify dimension tables and their categorical columns
    dim_tables_query = """
    SELECT TABLE_NAME, COLUMN_NAME
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME IN (
        SELECT TABLE_NAME 
        FROM INFORMATION_SCHEMA.TABLES 
        WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_NAME LIKE 'Dim%'
    )
    AND DATA_TYPE IN ('varchar', 'nvarchar', 'char', 'text')
    """
    
    dim_columns_df = pd.read_sql(dim_tables_query, conn)
    
    # Step 3: Filter out non-English columns and columns containing specified substrings
    non_english_pattern = re.compile(r'[^\x00-\x7F]+')

    # Function to check if a column name contains non-English characters or specified substrings
    def is_valid_column(column_name):
        substrings_to_remove = ['Description', 'FirstName', 'MiddleName', 'PhoneNumber', 'Phone' 'LastName', 'Address', 'ID', 'Key']
        if non_english_pattern.search(column_name):
            return False
        if any(substring in column_name for substring in substrings_to_remove):
            return False
        return True

    # Filter valid columns
    dim_columns_df = dim_columns_df[dim_columns_df['COLUMN_NAME'].apply(is_valid_column)]

    # Step 4: Process each fact table and dynamically join with dimension tables
    for file in os.listdir(all_final_outliers_dir):
        if file.endswith("_Only_Final_Outliers.csv"):
            # Extract the fact table name from the file
            table = file.replace("_Only_Final_Outliers.csv", "")
            agreed_outlier_file_path = os.path.join(all_final_outliers_dir, file)
            
            if os.path.exists(agreed_outlier_file_path):
                # Load the final outliers CSV file
                outlier_data = pd.read_csv(agreed_outlier_file_path)

                # Dynamically join outlier data with each dimension table based on foreign key
                for dim_table, dim_column in dim_columns_df.groupby('TABLE_NAME'):
                    categorical_columns = dim_column['COLUMN_NAME'].tolist()
                    
                    # Generate the query to get the dimension data
                    dim_data_query = f"SELECT * FROM {dim_table}"
                    dim_data = pd.read_sql(dim_data_query, conn)

                    # Determine the common foreign key column
                    foreign_keys = set(outlier_data.columns).intersection(set(dim_data.columns))
                    if foreign_keys:
                        foreign_key = foreign_keys.pop()
                        
                        # Merge outlier data with dimension table data on the foreign key
                        merged_data = pd.merge(outlier_data, dim_data, left_on=foreign_key, right_on=foreign_key, how='inner')

                        # Group by combinations of two categorical columns
                        for cat_col1, cat_col2 in combinations(categorical_columns, 2):
                            aggregated_data = merged_data.groupby([cat_col1, cat_col2]).size().reset_index(name='OutlierCount')

                            # Save aggregated results to CSV
                            aggregated_output_path = os.path.join(aggregated_dir, f"{table}_Aggregated_By_{dim_table}_{cat_col1}_{cat_col2}.csv")
                            aggregated_data.to_csv(aggregated_output_path, index=False)
                            print(f"Stored aggregated results for {table} by {dim_table}.{cat_col1} and {cat_col2} in {aggregated_output_path}.")
                    else:
                        print(f"No common foreign key found for table {table} and dimension table {dim_table}. Skipping...")
            else:
                print(f"No agreed outliers file found for {table}. Skipping...")

    # Close the database connection
    conn.close()
else:
    print("Database connection failed.")


Connected Successfully
No common foreign key found for table FactCallCenter and dimension table DimAccount. Skipping...
No common foreign key found for table FactCallCenter and dimension table DimCurrency. Skipping...


C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1940784503.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_columns_df = pd.read_sql(dim_tables_query, conn)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1940784503.py:73: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data = pd.read_sql(dim_data_query, conn)


No common foreign key found for table FactCallCenter and dimension table DimCustomer. Skipping...
Stored aggregated results for FactCallCenter by DimDate.EnglishDayNameOfWeek and SpanishDayNameOfWeek in Outlier_Detection_Test_9/Aggregated_By_Two_Category\FactCallCenter_Aggregated_By_DimDate_EnglishDayNameOfWeek_SpanishDayNameOfWeek.csv.
Stored aggregated results for FactCallCenter by DimDate.EnglishDayNameOfWeek and FrenchDayNameOfWeek in Outlier_Detection_Test_9/Aggregated_By_Two_Category\FactCallCenter_Aggregated_By_DimDate_EnglishDayNameOfWeek_FrenchDayNameOfWeek.csv.
Stored aggregated results for FactCallCenter by DimDate.EnglishDayNameOfWeek and EnglishMonthName in Outlier_Detection_Test_9/Aggregated_By_Two_Category\FactCallCenter_Aggregated_By_DimDate_EnglishDayNameOfWeek_EnglishMonthName.csv.
Stored aggregated results for FactCallCenter by DimDate.EnglishDayNameOfWeek and SpanishMonthName in Outlier_Detection_Test_9/Aggregated_By_Two_Category\FactCallCenter_Aggregated_By_DimDate

C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1940784503.py:73: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data = pd.read_sql(dim_data_query, conn)


No common foreign key found for table FactCallCenter and dimension table DimProduct. Skipping...
No common foreign key found for table FactCallCenter and dimension table DimProductCategory. Skipping...
No common foreign key found for table FactCallCenter and dimension table DimProductSubcategory. Skipping...
No common foreign key found for table FactCallCenter and dimension table DimPromotion. Skipping...
No common foreign key found for table FactCallCenter and dimension table DimReseller. Skipping...
No common foreign key found for table FactCallCenter and dimension table DimSalesReason. Skipping...
No common foreign key found for table FactCallCenter and dimension table DimSalesTerritory. Skipping...
No common foreign key found for table FactCallCenter and dimension table DimScenario. Skipping...
No common foreign key found for table FactCurrencyRate and dimension table DimAccount. Skipping...


...

Stored aggregated results for FactResellerSales by DimGeography.City and StateProvinceCode in Outlier_Detection_Test_9/Aggregated_By_Two_Category\FactResellerSales_Aggregated_By_DimGeography_City_StateProvinceCode.csv.
Stored aggregated results for FactResellerSales by DimGeography.City and StateProvinceName in Outlier_Detection_Test_9/Aggregated_By_Two_Category\FactResellerSales_Aggregated_By_DimGeography_City_StateProvinceName.csv.
Stored aggregated results for FactResellerSales by DimGeography.City and CountryRegionCode in Outlier_Detection_Test_9/Aggregated_By_Two_Category\FactResellerSales_Aggregated_By_DimGeography_City_CountryRegionCode.csv.
Stored aggregated results for FactResellerSales by DimGeography.City and EnglishCountryRegionName in Outlier_Detection_Test_9/Aggregated_By_Two_Category\FactResellerSales_Aggregated_By_DimGeography_City_EnglishCountryRegionName.csv.
Stored aggregated results for FactResellerSales by DimGeography.City and SpanishCountryRegionName in Outlier_D

C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1940784503.py:73: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data = pd.read_sql(dim_data_query, conn)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1940784503.py:73: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data = pd.read_sql(dim_data_query, conn)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1940784503.py:73: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data = pd.read_sql(dim_data_query, conn)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\194078

Stored aggregated results for FactResellerSales by DimReseller.ResellerName and ProductLine in Outlier_Detection_Test_9/Aggregated_By_Two_Category\FactResellerSales_Aggregated_By_DimReseller_ResellerName_ProductLine.csv.
Stored aggregated results for FactResellerSales by DimReseller.ResellerName and BankName in Outlier_Detection_Test_9/Aggregated_By_Two_Category\FactResellerSales_Aggregated_By_DimReseller_ResellerName_BankName.csv.
Stored aggregated results for FactResellerSales by DimReseller.OrderFrequency and ProductLine in Outlier_Detection_Test_9/Aggregated_By_Two_Category\FactResellerSales_Aggregated_By_DimReseller_OrderFrequency_ProductLine.csv.
Stored aggregated results for FactResellerSales by DimReseller.OrderFrequency and BankName in Outlier_Detection_Test_9/Aggregated_By_Two_Category\FactResellerSales_Aggregated_By_DimReseller_OrderFrequency_BankName.csv.
Stored aggregated results for FactResellerSales by DimReseller.ProductLine and BankName in Outlier_Detection_Test_9/Aggr

C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1940784503.py:73: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data = pd.read_sql(dim_data_query, conn)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_47616\1940784503.py:73: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_data = pd.read_sql(dim_data_query, conn)


Stored aggregated results for FactSalesQuota by DimEmployee.Title and DepartmentName in Outlier_Detection_Test_9/Aggregated_By_Two_Category\FactSalesQuota_Aggregated_By_DimEmployee_Title_DepartmentName.csv.
Stored aggregated results for FactSalesQuota by DimEmployee.Title and Status in Outlier_Detection_Test_9/Aggregated_By_Two_Category\FactSalesQuota_Aggregated_By_DimEmployee_Title_Status.csv.
Stored aggregated results for FactSalesQuota by DimEmployee.Phone and EmergencyContactName in Outlier_Detection_Test_9/Aggregated_By_Two_Category\FactSalesQuota_Aggregated_By_DimEmployee_Phone_EmergencyContactName.csv.
Stored aggregated results for FactSalesQuota by DimEmployee.Phone and EmergencyContactPhone in Outlier_Detection_Test_9/Aggregated_By_Two_Category\FactSalesQuota_Aggregated_By_DimEmployee_Phone_EmergencyContactPhone.csv.
Stored aggregated results for FactSalesQuota by DimEmployee.Phone and DepartmentName in Outlier_Detection_Test_9/Aggregated_By_Two_Category\FactSalesQuota_Aggrega

**Aggregation Results Count**

In [4]:
import os
import pandas as pd

# List of folder paths to check
folder_paths = ['Outlier_Detection_Test_9/Aggregated_By_One_Category', 'Outlier_Detection_Test_9/Aggregated_By_Two_Category']  # Replace with your actual folder paths

# List to store folder names and CSV file counts
folder_info = []

# Loop through each folder path
for folder_path in folder_paths:
    # Count the number of CSV files in the current folder
    csv_count = sum([1 for file_name in os.listdir(folder_path) if file_name.endswith('.csv')])
    
    # Store the folder path and count in the list
    folder_info.append({'Folder Path': folder_path, 'CSV File Count': csv_count})

# Convert the list to a DataFrame and display it
folder_info_df = pd.DataFrame(folder_info)
print(folder_info_df)


                                         Folder Path  CSV File Count
0  Outlier_Detection_Test_9/Aggregated_By_One_Cat...             139
1  Outlier_Detection_Test_9/Aggregated_By_Two_Cat...             451
